In [93]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely import wkt
import boto3
import botocore
import datetime
import matplotlib.pyplot as plt
import os.path
import xarray as xr
import folium
import folium.plugins as plugins
import numpy as np

# We have our data in the df
helena = pd.read_csv('data/helena_tehsils.csv')

# view and merge files
helena = pd.read_csv('helena_tehsils_correct.csv')
helena['Correct_Tehsil'] = helena['Correct_Tehsil'].apply(lambda x: x.lower())

# Get the polygon shapes
table = gpd.read_file('coordinates/adm3/geoBoundaries-PAK-ADM3.dbf')
df_adm3 = pd.DataFrame(table)

polygons = df_adm3[['shapeName', 'geometry']]
polygons['shapeName'] = polygons['shapeName'].apply(lambda x: x.lower())

# Select Tehsils of interest
df_adm3['shapeName'] = df_adm3['shapeName'].apply(lambda x: x.lower())
common = list(set(helena['Correct_Tehsil']).intersection(df_adm3['shapeName']))
teh_interest = df_adm3[df_adm3['shapeName'].isin(common)]



/var/folders/ml/57p2p5r56p35n0ttkg342kmw0000gn/T/ipykernel_31694/3181779995.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  polygons['shapeName'] = polygons['shapeName'].apply(lambda x: x.lower())


In [94]:
# Now we append the points for each tehsil
centroids = pd.read_csv('df_centroids.csv')
# turning into GeoPandasDF
centroids['point'] = centroids['point'].apply(wkt.loads)
centroids = gpd.GeoDataFrame(centroids)
# selecting the points column
points = centroids['point']

In [95]:
# for point in points:
teh_interest['points'] = teh_interest['geometry'].apply(lambda x: [point for point in points if x.contains(point) == True])

/var/folders/ml/57p2p5r56p35n0ttkg342kmw0000gn/T/ipykernel_31694/819702046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teh_interest['points'] = teh_interest['geometry'].apply(lambda x: [point for point in points if x.contains(point) == True])


In [98]:
era5_bucket = 'era5-pds'
# Without AWS key
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))
# Available Years
# paginator = client.get_paginator('list_objects')
# result = paginator.paginate(Bucket=era5_bucket, Delimiter='/')
# for prefix in result.search('CommonPrefixes'):
#     print(prefix.get('Prefix'))

In [ ]:
# Downloading variables of interest
air_temp = 'air_temperature_at_2_metres'
air_temp_max = 'air_temperature_at_2_metres_1hour_Maximum'
air_temp_min = 'air_temperature_at_2_metres_1hour_Minimum'
precip = 'precipitation_amount_1hour_Accumulation'

def request_era5(location, var, date):
    files = []
    year = date.strftime('%Y')
    month = date.strftime('%m')
    # s3 file path
    s3_path ='{year}/{month}/data/{var}.nc'
    file_path = 'era5_data/{location}/{year}{month}_{var}.nc'

    s3_data_key = s3_path.format(year=year, month=month, var=var)
    data_file = file_path.format(year=year, month=month, var=var, location=location)
    if not os.path.isfile(data_file): # check if file already exists
        print("Downloading %s from S3..." % s3_data_key)
        client.download_file(era5_bucket, s3_data_key, data_file)
    files.append(data_file)
    return files
# for i in range(1, 13):
#     date = datetime.date(2022,i,1) # update to desired date
#     request_era5('precip', precip, date )

In [96]:
# double checking if the function worked
import folium
m_poly = teh_test.iloc[0]['geometry']
point_test = teh_test.iloc[0]['points']
m = folium.Map([29.929705600000023, 72.997855], zoom_start=10)
folium.GeoJson(m_poly).add_to(m)
folium.Marker([29.93, 72.998]).add_to(m)
folium.Marker([29.68, 73.248]).add_to(m)
folium.Marker([29.93, 73.248]).add_to(m)
m